In [16]:
import MySQLdb
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

aurora_host = os.getenv('AURORA_HOST')
aurora_database = os.getenv('AURORA_DATABASE')
aurora_user = os.getenv('AURORA_USER')
aurora_pwd = os.getenv('AURORA_PWD')
conn = MySQLdb.connect(host=aurora_host, user=aurora_user, password=aurora_pwd, db=aurora_database, charset='utf8', port=3306)
cur = conn.cursor()

In [3]:
# 상권 번호로 검색 시 연도별 총 매출
cur.execute(
    '''
    SELECT cd_cd, SUBSTRING(aq_cd, 1, 4) AS year, 
        SUM(cm_sa) AS total_sales
    FROM cd_sales
    WHERE cd_cd = 3110008
    GROUP BY SUBSTRING(aq_cd, 1, 4);
    '''
)

conn.commit()
conn.close()

In [ ]:
# 행정동 번호로 검색 시 연도별 총 매출
cur.execute(
    '''
    SELECT r.ab_cd,SUBSTRING(s.aq_cd, 1, 4) AS year, 
        SUM(s.cm_sa) AS total_sales
    FROM cd_sales s
    JOIN region_info r ON s.cd_cd = r.cd_cd
    WHERE r.ab_cd = 11110515
    GROUP BY SUBSTRING(s.aq_cd, 1, 4);
    '''
)

conn.commit()
conn.close()

In [ ]:
# 행정동 번호별 상권별 연도별로 가장 매출이 높은 업종과 총 매출 출력
cur.execute(
    '''
    SELECT *
    FROM
        (SELECT r.ab_cd, 
                s.cd_cd, 
                SUBSTRING(s.aq_cd, 1, 4) AS year,
                SUM(s.cm_sa) AS total_cm_sa,
                s.st_cd,
                RANK() OVER (PARTITION BY s.cd_cd, SUBSTRING(s.aq_cd, 1, 4) ORDER BY SUM(s.cm_sa) DESC) AS rnk
        FROM cd_sales s
        JOIN region_info r ON s.cd_cd = r.cd_cd
        WHERE r.ab_cd = 11170650
        GROUP BY r.ab_cd, s.cd_cd, year, s.st_cd) as t1
    WHERE t1.rnk = 1;
    '''
)

conn.commit()
conn.close()

In [ ]:
# 행정동 번호별 상권별 연도별 총 매출을 출력
cur.execute(
    '''
    SELECT t2.ab_cd, t2.cd_cd, year, sum(t2.total_cm_sa) FROM
        (SELECT r.ab_cd, 
                s.cd_cd, 
                SUBSTRING(s.aq_cd, 1, 4) AS year,
                SUM(s.cm_sa) AS total_cm_sa,
                s.st_cd,
                RANK() OVER (PARTITION BY s.cd_cd, SUBSTRING(s.aq_cd, 1, 4) ORDER BY SUM(s.cm_sa) DESC) AS rnk
        FROM cd_sales s
        JOIN region_info r ON s.cd_cd = r.cd_cd
        WHERE r.ab_cd = 11170650
        GROUP BY r.ab_cd, s.cd_cd, year, s.st_cd) as t2
    GROUP BY t2.ab_cd, t2.cd_cd, year;
    '''
)

conn.commit()
conn.close()

In [ ]:
# 행정동 번호별 상권별 연도별 총 매출과 해당 상권의 해당 연도에 가장 매출이 높은 업종의 업종명, 총 매출, 비율을 출력하는 view 생성
cur.execute(
    '''
    CREATE VIEW ab_summary_view AS
    SELECT t5.ab_cd, t5.ab_nm, t5.cd_cd, t5.cd_nm, t5.year, t5.cd_total_sa, i.st_nm, t5.ratio
    FROM
        (SELECT t1.ab_cd, t1.ab_nm, t1.cd_cd, t1.cd_nm,
            t1.year, t3.total_cm_sa as cd_total_sa, t1.total_cm_sa as sector_total_sa, t1.st_cd, (t1.total_cm_sa /  t3.total_cm_sa)*100 as ratio
        FROM
            (SELECT r.ab_cd, r.ab_nm,
                s.cd_cd, r.cd_nm,
                SUBSTRING(s.aq_cd, 1, 4) AS year,
                SUM(s.cm_sa) AS total_cm_sa,
                s.st_cd,
                RANK() OVER (PARTITION BY s.cd_cd, SUBSTRING(s.aq_cd, 1, 4) ORDER BY SUM(s.cm_sa) DESC) AS rnk
            FROM cd_sales s
            JOIN region_info r ON s.cd_cd = r.cd_cd
            GROUP BY r.ab_cd, s.cd_cd, year, s.st_cd) as t1
        JOIN
            (SELECT t2.ab_cd, t2.cd_cd, year, sum(t2.total_cm_sa) as total_cm_sa FROM
                (SELECT r.ab_cd,
                    s.cd_cd,
                    SUBSTRING(s.aq_cd, 1, 4) AS year,
                    SUM(s.cm_sa) AS total_cm_sa,
                    s.st_cd,
                    RANK() OVER (PARTITION BY s.cd_cd, SUBSTRING(s.aq_cd, 1, 4) ORDER BY SUM(s.cm_sa) DESC) AS rnk
                    FROM cd_sales s
                    JOIN region_info r ON s.cd_cd = r.cd_cd
                    GROUP BY r.ab_cd, s.cd_cd, year, s.st_cd) as t2
            GROUP BY t2.ab_cd, t2.cd_cd, year) as t3 ON t1.ab_cd = t3.ab_cd AND t1.cd_cd = t3.cd_cd AND t1.year = t3.year
        WHERE t1.rnk = 1
        GROUP BY t1.ab_cd, t1.cd_cd, t1.year, t3.total_cm_sa, t1.total_cm_sa, t1.st_cd) as t5
    JOIN code_info i on t5.st_cd = i.st_cd;
    '''
)

conn.commit()
conn.close()

# view 테이블에서 행정동 번호로 검색
# ab_cd: 행정동,
# ab_nm: 행정동명,
# cd_cd: 상권,
# cd_nm: 상권명,
# year: 연도, 
# cd_total_sa: 해당 상권의 해당 연도 총 매출, 
# st_nm: 해당 상권의 해당 연도에 최고 매출을 기록한 업종명
# ratio: 최고 매출을 기록한 업종의 상권 총 매출 대비 비율

In [17]:
# 행정동 클릭 시 쿼리 발생
cur.execute(
    '''
    SELECT * FROM ab_summary_view
    WHERE ab_cd = 11170650
    '''
)

25

In [18]:
# 결과 가져오기
results = cur.fetchall()

# 결과를 DataFrame으로 변환
df = pd.DataFrame(results, columns=[desc[0] for desc in cur.description])

# 연결과 커서 닫기
cur.close()
conn.close()

In [19]:
# DataFrame 출력
df.head(5)

,ab_cd,ab_nm,cd_cd,cd_nm,year,cd_total_sa,st_nm,ratio
0,11170650,이태원1동,3001491,이태원 관광특구,2019,369839184688,일반의류,18.2503
1,11170650,이태원1동,3001491,이태원 관광특구,2020,276297362501,일반의류,15.8586
2,11170650,이태원1동,3001491,이태원 관광특구,2021,288195031461,양식음식점,16.2457
3,11170650,이태원1동,3001491,이태원 관광특구,2022,397416392896,양식음식점,21.8355
4,11170650,이태원1동,3001491,이태원 관광특구,2023,265351032699,양식음식점,19.5096
